In [24]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler goatools gseapy scperturb chembl_webresource_client biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

In [25]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns
import scvelo as scv
scv.settings.verbosity=1

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../../')
from utils import *

# scperturb package
sys.path.insert(1, '../../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../../figures/')

In [26]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')
DATADIR = Path('/home/peidli/data/scPerturb/')

In [27]:
from scipy.io import mmread
from scipy.sparse import csr_matrix

In [28]:
files = [x.name for x in (TEMPDIR / 'SantinhaPlatt2023').glob('*_temp.h5ad')]

In [29]:
files

['SantinhaPlatt2023_GSE236519_arrayed_screen_hSyn_temp.h5ad',
 'SantinhaPlatt2023_GSE236519_lgdel_model_temp.h5ad',
 'SantinhaPlatt2023_GSE236519_pooled_screen_CBh_temp.h5ad',
 'SantinhaPlatt2023_GSE236519_arrayed_CRISPRi_hSyn_temp.h5ad']

In [44]:
# main dataset
adata = sc.read(TEMPDIR / 'SantinhaPlatt2023' / 'SantinhaPlatt2023_GSE236519_pooled_screen_CBh_temp.h5ad')

# obs
adata.obs.set_index('Barcode', drop=True, inplace=True)
adata.obs.index.name = 'cell_barcode'
adata.obs['organism'] = 'Mus musculus'
adata.obs['disease'] = 'healthy'
adata.obs['cancer'] = False
adata.obs['perturbation_type'] = 'CRISPR-cas9'
adata.obs['tissue_type'] = 'primary'
adata.obs['tissue'] = 'brain'
adata.obs.rename({
    'per_gene': 'perturbation',
    'nCount_RNA':'ncounts',
    'nFeature_RNA': 'ngenes',
    'gRNAs': 'guide_id'
}, axis=1, inplace=True)
adata.obs.perturbation = adata.obs.perturbation.astype(str)
adata.obs.perturbation[adata.obs.perturbation=='Safe_H'] = 'control'

adata.obs['nperts'] = [1-p.count('control') if type(p)==str else None for p in adata.obs.perturbation]


# var
adata.var.index.name = 'gene_symbol'

annotate_qc(adata, species='mouse')
assert_annotations(adata)